<a href="https://colab.research.google.com/github/protoi/Speech-Emotion-Analysis/blob/main/4_Arranging_Spectrogram_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING LIBRARIES

In [ ]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as math
import librosa as lib
import librosa.display as lid
import re 
import os as os
import glob as glob
from pathlib import Path
from os import chdir
from scipy.signal import butter, filtfilt
import gc
from tqdm import tqdm
from google.colab import drive
import shutil
from google.colab import output
import IPython
from IPython.display import Audio
import random as rand
from pylab import Rectangle
import gc 
from scipy import stats
import soundfile as sf
import pyarrow.feather as feather
from scipy.signal import butter
from scipy.signal import butter, filtfilt
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image, ImageEnhance
from tqdm.auto import tqdm
import random

# OBTAINING FILES FROM GOOGLE DRIVE

## Copying zipped files from colab and unzipping them to /content/spectrograms


In [ ]:
!cp /content/drive/MyDrive/Machine\ Learning/Speech\ Emotion\ Recognition/spectrograms_30_09_2022.zip /content/spectrograms.zip
!unzip spectrograms.zip 

Streaming output truncated to the last 5000 lines.
  inflating: spectrograms/male_anger-CREMAD-1023_ITH_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_ITS_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_IWL_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_IWW_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_MTI_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_TAI_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_TIE_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_TSI_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1023_WSI_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1026_DFA_ANG_XX.png  
  inflating: spectrograms/male_anger-CREMAD-1026_IEO_ANG_HI.png  
  inflating: spectrograms/male_anger-CREMAD-1026_IEO_ANG_LO.png  
  inflating: spectrograms/male_anger-CREMAD-1026_IEO_ANG_MD.png  
  inflating: spectrograms/male_anger-CREMAD-1026_IOM_ANG_XX.png  
  inflating: spectrograms

# ARRANGING DATA FOR /content/input_data

### LISTING THE DIRECTORY CONTENTS

In [ ]:
os.listdir('/content/spectrograms')

['male_neutral-CREMAD-1081_TAI_NEU_XX.png',
 'male_fear-CREMAD-1087_TSI_FEA_XX.png',
 'male_surprise-SAVEE-JE_su06.png',
 'female_neutral-CREMAD-1030_DFA_NEU_XX.png',
 'female_sadness-RAVDESS-Actor_04_03-01-04-01-01-02-04.png',
 'female_disgust-TESS-YAF_tip_disgust.png',
 'female_disgust-TESS-OAF_youth_disgust.png',
 'female_happy-TESS-YAF_said_happy.png',
 'female_fear-TESS-YAF_join_fear.png',
 'male_fear-CREMAD-1087_IWL_FEA_XX.png',
 'female_sadness-TESS-YAF_wag_sad.png',
 'male_sadness-CREMAD-1083_ITH_SAD_XX.png',
 'female_anger-CREMAD-1089_TIE_ANG_XX.png',
 'male_fear-CREMAD-1044_IOM_FEA_XX.png',
 'female_neutral-CREMAD-1037_IEO_NEU_XX.png',
 'male_sadness-CREMAD-1081_TSI_SAD_XX.png',
 'female_disgust-TESS-OAF_raid_disgust.png',
 'female_calm-RAVDESS-Actor_18_03-01-02-01-02-01-18.png',
 'male_disgust-CREMAD-1023_IWL_DIS_XX.png',
 'male_sadness-CREMAD-1034_IEO_SAD_MD.png',
 'female_fear-TESS-OAF_south_fear.png',
 'female_sadness-CREMAD-1006_ITH_SAD_XX.png',
 'male_surprise-SAVEE-JE_

### MAKING A SET OUT OF EMOTIONS FOR THE SUB-FOLDER STRUCTURE
---
#### for example ```male_anger``` or ```female_sadness``` 

In [ ]:
name_list = os.listdir('/content/spectrograms/')

file_detector = re.compile(r"([\w]+)-([\w]+)-(.*)\.png$") #emotion-dataset-filename.png      group(1) -> emotion

directory_list = list(map(lambda x: file_detector.search(x).group(1), name_list))

directory_list_set = set(directory_list) #contains unique directories(emotions)


### MAKING A DICTIONARY OUT OF THE SET TO STORE PATHS FOR EACH EMOTIONS AS A LIST 
---
#### for example 
```python
dict['male_anger'] = ['123.png','abcd.png','xyz.png']
dict['female_anger'] = ['456.png','pqr.png','lmnop.png']
```

In [ ]:
directory_list_dict = dict.fromkeys(directory_list_set)
directory_list_dict

for name, dir in zip(name_list ,directory_list):
  if directory_list_dict[dir] == None:
    directory_list_dict[dir] = []
  directory_list_dict[dir].append(name)

In [ ]:
#@title just a visual representation of each emotion and it's file count
for k, v in directory_list_dict.items():
  print(f'{k} -> {len(v)}')

female_neutral -> 960
male_sadness -> 827
female_sadness -> 1095
male_calm -> 96
male_happy -> 827
male_neutral -> 743
female_happy -> 1096
female_calm -> 96
male_anger -> 827
male_disgust -> 827
female_disgust -> 1096
female_anger -> 1096
female_surprise -> 496
male_surprise -> 156
male_fear -> 827
female_fear -> 1096


In [ ]:
#@title splits a list into 2 random parts using a ratio 
def split_into_sections(list_to_split, break_point_1, break_point_2): #split_ratio has to be a value between 0 and 1
  random.shuffle(list_to_split)
  mid_point_1 = math.floor(len(list_to_split) * break_point_1)
  mid_point_2 = math.floor(len(list_to_split) * break_point_2)
  return (list_to_split[ : mid_point_1], 
          list_to_split[ mid_point_1 : mid_point_2], 
          list_to_split[mid_point_2 : ])

In [ ]:
#@title using this to generate a dictionary for the file paths later
directory_list_set_test_train = set()

for emo in directory_list_set:
  directory_list_set_test_train.add(f'TEST_{emo}')
  directory_list_set_test_train.add(f'TRAIN_{emo}')
  directory_list_set_test_train.add(f'VALIDATION_{emo}')


directory_list_dict_test_train_split = dict.fromkeys(directory_list_set_test_train)

In [ ]:
#@title making a dictionary for the file paths to different emotions and randomly splitting them for training and testing with a specified ratio

for k, v in directory_list_dict.items():
  train_data, validation_data, test_data = split_into_sections(v, 0.8, 0.9)
  directory_list_dict_test_train_split[f'TRAIN_{k}'] = train_data
  directory_list_dict_test_train_split[f'TEST_{k}'] = test_data
  directory_list_dict_test_train_split[f'VALIDATION_{k}'] = validation_data

### MAKING THE FILE STRUCTURE TO PUT THE IMAGES IN

In [ ]:
if os.path.exists('/content/input_data'):
  shutil.rmtree('/content/input_data')

os.mkdir(f'/content/input_data')

for directory_name in directory_list_set_test_train:
  if os.path.exists(f'/content/input_data/{directory_name}'):
    shutil.rmtree(f'/content/input_data/{directory_name}')
  os.mkdir(f'/content/input_data/{directory_name}')

### MOVING THE UNORGANIZED FILES FROM /content/spectrograms 
### to /content/input INTO THE CORRECT TEST_ or TRAIN_ EMOTION SUB-DIRECTORY 

In [ ]:
for k, v in directory_list_dict_test_train_split.items(): # k is the emotion (destination sub folder) and v is a list containing paths
  for paths in v:                                         # paths is the name of a singular file in /content/spectrograms 
    source_dir = f'/content/spectrograms/{paths}'
    destination_dir = f'/content/input_data/{k}/{paths}'
    # pass
    shutil.move(source_dir, destination_dir)

In [ ]:
#@title freeing up data from unused variables
del(v)
del(test_data)
del(train_data)
del(directory_list)
del(directory_list_dict)
del(directory_list_dict_test_train_split)
del(directory_list_set)
del(directory_list_set_test_train)
del(name_list)